In [3]:
from dotenv import load_dotenv
import os

from pydantic import BaseModel, Field
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

In [ ]:
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY") 
print(api_key[:5])

gsk_4


In [4]:
class StudentInfo(BaseModel):
    name: str = Field(description="학생의 이름")
    age: int = Field(description="학생의 나이")
    major: str = Field(description="학생의 전공")
    hobbies: list[str] = Field(description="학생의 취미 리스트)")
    goal: str = Field(description="학생의 목표")

parser = PydanticOutputParser(pydantic_object=StudentInfo)
format_instructions = parser.get_format_instructions() 

prompt = PromptTemplate(
    template=(
        "아래 학생의 자유 형식 자기소개에서 정보를 추출하여 JSON으로 반환하세요.\n"
        "반드시 아래 형식과 타입을 따르세요.\n\n"
        "{format_instructions}\n\n"
        "자유 형식 자기소개:\n{text}"
    ),
    input_variables=["text"],
    partial_variables={"format_instructions": format_instructions},
)

llm = ChatOpenAI(
    api_key=api_key,
    base_url="https://api.groq.com/openai/v1",
    model="moonshotai/kimi-k2-instruct-0905",
    temperature=0.7,  
)

chain = prompt | llm | parser

student_text = (
    "안녕하세요! 저는 김민수이고 22살입니다. "
    "컴퓨터공학을 전공하고 있어요. "
    "취미로는 게임하기, 영화보기, 코딩을 좋아합니다. "
    "앞으로 훌륭한 개발자가 되는 것이 목표입니다."
)

structured = chain.invoke({"text": student_text})

print("--- 추출된 학생 정보 ---")
print(f"이름: {structured.name}")
print(f"나이: {structured.age}")
print(f"전공: {structured.major}")
print(f"취미: {structured.hobbies}")
print(f"목표: {structured.goal}")

print("\n--- 결과 타입 ---")
print(type(structured))

--- 추출된 학생 정보 ---
이름: 김민수
나이: 22
전공: 컴퓨터공학
취미: ['게임하기', '영화보기', '코딩']
목표: 훌륭한 개발자가 되는 것

--- 결과 타입 ---
<class '__main__.StudentInfo'>
